# Saudi Point Of sales Transaction dashboard

In [1]:
# import libraries
import pandas as pd
import numpy as np
import glob


import pdfplumber
import re
from dfhelper import *
from datetime import datetime
from dateutil import parser 


import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from fake_useragent import UserAgent
from geopy.geocoders import Nominatim

In [ ]:
#Monthly pos transaction
#first page
url = "https://www.sama.gov.sa/ar-sa/EconomicReports/pages/monthlystatistics.aspx"

# sent off request object to a server to request and query some resource aas.
with requests.Session() as s:

    # The User-Agent request header is a characteristic string that lets servers and network peers identify the application, operating system, vendor, and/or version of the requesting user agent.
    s.headers = {'User-Agent': 'Mozilla/5.0 '}

    #Response object is generated once Requests gets a response back from the server. The Response object contains all of the information returned by the server and also contains the Request object we created originally
    response = s.get(url)

    #pulling data out of HTML using BeautifulSoup
    soup = BeautifulSoup(response.content)
    #FileRef = re.findall("xlsx", str(soup), re.MULTILINE | re.DOTALL )

    FileRef = re.findall("FileRef.*?[\.!?]", str(soup), re.MULTILINE | re.DOTALL )

    #fix url using .replace method
    FileRefNew = [item.replace("\\u002far-sa\\u002fEconomicReports\\u002fMonthlyStatistics\\u002f", "https://www.sama.gov.sa/ar-sa/EconomicReports/MonthlyStatistics/") for item in FileRef]
    string = 'xlsx'
    #adding .pdf to the end of url
    FileRefNew = [x + string for x in FileRefNew]
    FileRefNew = [e[11:] for e in FileRefNew]
    print(FileRefNew[0])


try:
    # Manage firefox specific settings in a way that geckodriver can understand 
    options = webdriver.FirefoxOptions()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", "./input")
    options.set_preference("browser.download.useDownloadDir", True)
    #options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/xls;text/csv")
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    # call selenium driver to automate web browser interaction
    driver = webdriver.Firefox(executable_path="./geckodriver", options = options)
    # open an URL
    driver.get(FileRefNew[0])
    driver.implicitly_wait(10)
    driver.close()
except WebDriverException:
    driver.quit()
    exit()

In [5]:
# Weekly pos transaction
#first page
url = "https://www.sama.gov.sa/ar-sa/Indices/pages/pos.aspx"

# sent off request object to a server to request and query some resource aas.
with requests.Session() as s:

    # The User-Agent request header is a characteristic string that lets servers and network peers identify the application, operating system, vendor, and/or version of the requesting user agent.
    s.headers = {'User-Agent': 'Mozilla/5.0 '}

    #Response object is generated once Requests gets a response back from the server. The Response object contains all of the information returned by the server and also contains the Request object we created originally
    response = s.get(url)

    #pulling data out of HTML using BeautifulSoup
    soup = BeautifulSoup(response.content)


    #The re.findall method scans string, searching for all matches and the pattern. It returns a list of strings in the matching order when scanning the string.
    # re.MULTILINE tag affects where ^ and $ anchors match.
    # re.DOTALL tag affects what the . pattern can match.
    FileRef = re.findall("FileRef.*?[\.!?]", str(soup), re.MULTILINE | re.DOTALL )

    # fix url using .replace method
    FileRefNew = [item.replace("\\u002far-sa\\u002fIndices\\u002fPOS\\u002f", "https://www.sama.gov.sa/ar-sa/Indices/POS/") for item in FileRef]
    string = 'pdf'
    #adding .pdf to the end of url
    FileRefNew = [x + string for x in FileRefNew]
    FileRefNew = [e[11:] for e in FileRefNew]
    print(FileRefNew[0])


https://www.sama.gov.sa/ar-sa/Indices/POS/Weekly_Points_of_Sale_Transactions_Report_9th-Apr-2022.pdf


In [9]:
# print(FileRefNew[0:6])
#check_last_pdf = pd.read_csv("./output/sectors_df.csv")
#check_last_pdf.Date.max()

'2021-11-20'

In [10]:
# Loop through all urls in FileRefNew and download pdf file
for url in FileRefNew:
    try:
        # Manage firefox specific settings in a way that geckodriver can understand 
        options = webdriver.FirefoxOptions()
        options.set_preference("browser.download.folderList", 2)
        options.set_preference("browser.download.manager.showWhenStarting", False)
        options.set_preference("browser.download.dir", "/Users/turki/Desktop/SaudiPointOfSales-staging/pdf")
        options.set_preference("browser.download.useDownloadDir", True)
        options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
        options.set_preference("pdfjs.disabled", True)
        options.set_preference("pdfjs.enabledCache.state", False)
        options.set_preference("plugin.scan.Acrobat", "99.0")
        options.set_preference("plugin.scan.plid.all", False)
        # call selenium driver to automate web browser interaction
        driver = webdriver.Firefox(executable_path="./geckodriver", options = options)
        # open an URL
        driver.get(url)
        driver.implicitly_wait(10)
        driver.close()
    except WebDriverException:
        driver.quit()
        exit()

<ipython-input-10-7f02a3913bbc>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path="./geckodriver", options = options)


KeyboardInterrupt: 

In [4]:
# fitch all pdf inside pdf folder using glob method 
arr_of_files = (glob.glob("./pdf/*.pdf"))

# Create or initialize Pandas DataFrame
sectors_df = pd.DataFrame()
cities_df = pd.DataFrame()

# loop through each pdf and start scraping tables from pdfs  & save it in dataframs     
for i in arr_of_files:
    # Plumb a PDF for detailed information and table and store it in the initialize datafram.
    with pdfplumber.open(i) as pdf:
        tables = pdf.pages[0].find_tables()
        first_table = tables[0].extract(x_tolerance = 5)
        if '2020' in i:
            second_table = tables[2].extract(x_tolerance = 5)
        else:
            second_table = tables[1].extract(x_tolerance = 5)
        
        # Do some data cleansing in the first table inside a pdf 
        df_first_table = pd.DataFrame (first_table)
        df_first_table = df_first_table[[0,7,8]]
        df_first_table['Date'] = df_first_table[7][0]
        df_first_table = df_first_table[2:]
        df_first_table = df_first_table.reset_index(drop=True)
        
        sectors_df = sectors_df.append(df_first_table)
        
        # Do some data cleansing in the second table inside a pdf 
        df_second_table = pd.DataFrame (second_table)
        df_second_table = df_second_table[[0,7,8]]
        df_second_table['Date'] = df_second_table[7][0]
        df_second_table = df_second_table[2:]
        df_second_table = df_second_table.reset_index(drop=True)
        cities_df = cities_df.append(df_second_table)
        
print("** Done converting tables to data frames **")

# Renames columns  
sectors_df = sectors_df.rename(columns={0: 'Sector',7: 'Number of Transactions', 8: 'Value of Transactions'})
cities_df = cities_df.rename(columns={0: 'City',7: 'Number of Transactions', 8: 'Value of Transactions'})
cities_df = cities_df.drop(cities_df[cities_df['Value of Transactions']=='Value of \nTransactions'].index)
# change the columns value type from string to intger by using to_int function from helper
sectors_df = sectors_df.dropna(how='any',axis=0)
to_float(cities_df,'Value of Transactions')
to_int(cities_df,'Number of Transactions')
to_float(sectors_df,'Value of Transactions')
to_int(sectors_df,'Number of Transactions')

# parsing dates

sectors_df["End Date"]= sectors_df['Date'].str.split("-", n = 1, expand = True)[1]
# if statment due to missing year in 2020 pdfs 
if ',21' in sectors_df["End Date"].values:
    pass
else:
    sectors_df["End Date"]= sectors_df["End Date"] + ',20'
sectors_df['End Date'] = [pd.to_datetime(x) for x in sectors_df['End Date']]
sectors_df['Date'] = sectors_df['End Date']
del sectors_df['End Date']


cities_df["End Date"]= cities_df['Date'].str.split("-", n = 1, expand = True)[1]
# if statment due to missing year in 2020 pdfs 
if ',21' in cities_df["End Date"].values:
    pass
else:
    cities_df["End Date"]= cities_df["End Date"] + ',20'
cities_df['End Date'] = [pd.to_datetime(x) for x in cities_df['End Date']]
cities_df['Date'] = cities_df['End Date']
del cities_df['End Date']

# Doing more data cleansing on city and sector columns
sectors(sectors_df)
cities(cities_df)


# Add latitude & longitude for map chart using Nominatim
group_City = cities_df.groupby(by='English_City').agg({'Value of Transactions' : 'sum', 'Number of Transactions' : 'sum'})
group_City = group_City.reset_index()
location = [x for x in group_City['English_City'].unique().tolist() 
            if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    # remove things that does not seem usefull here
    try:
        address = location[i] + ', Saudi Arabia'
        geolocator = Nominatim(user_agent="sa_explorer@gmail.com")
        loc = geolocator.geocode(address)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
        print('The geographical coordinate of location are {}, {}.'.format(loc.latitude, loc.longitude))
    except:
        # in the case the geolocator does not work, then add nan element to list
        # to keep the right size
        latitude.append(np.nan)
        longitude.append(np.nan)
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'English_City':location, 
                    'location_latitude': latitude,
                    'location_longitude':longitude})
# merge on English_City with Groupe_City to get the column 
Grouped_City = group_City.merge(df_, on='English_City', how='left')
Grouped_City.at[Grouped_City['English_City'] == 'OTHER','location_latitude'] = float(25)
Grouped_City.at[Grouped_City['English_City'] == 'OTHER','location_longitude'] = float(45)

# change the order of dfs columns & export it as csv
columnsTitles = ['English_Sector', 'Arabic_Sector','Date', 'Number of Transactions', 'Value of Transactions']
sectors_df = sectors_df.reindex(columns=columnsTitles)
columnsTitles = ['English_City', 'Arabic_City','Date', 'Number of Transactions', 'Value of Transactions']
cities_df = cities_df.reindex(columns=columnsTitles)

sectors_df.to_csv('output/sectors_df.csv', index = False)
# merge cities_df and Grouped_City in order to standerlize df, it we help us alot once we plot our data
full_cities_df = pd.merge(cities_df, Grouped_City[['English_City', 'location_latitude', 'location_longitude']], on='English_City')
full_cities_df.to_csv('output/full_cities_df.csv', index = False)

print("** Done cleansing data frames **")

** Done converting tables to data frames **


/Users/turki/Desktop/SaudiPointOfSales-staging/dfhelper/__init__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['English_Sector'][df.English_Sector.str.contains('Clothing')] = 'Clothing and Footwear'
/Users/turki/Desktop/SaudiPointOfSales-staging/dfhelper/__init__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['English_Sector'][df.English_Sector.str.contains('Construction')] = 'Construction & Building Materials'
/Users/turki/Desktop/SaudiPointOfSales-staging/dfhelper/__init__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

The geographical coordinate of location are 18.2164282, 42.5043596.
The geographical coordinate of location are 26.331667, 43.971667.
The geographical coordinate of location are 26.4367824, 50.1039991.
The geographical coordinate of location are 27.828949899999998, 40.81830649689581.
The geographical coordinate of location are 21.5810088, 39.1653612.
The geographical coordinate of location are 26.3039999, 50.1960237.
The geographical coordinate of location are 25.1354363, 39.162594.
The geographical coordinate of location are 21.420847, 39.826869.
The geographical coordinate of location are 24.638916, 46.7160104.
The geographical coordinate of location are 28.4012536, 36.567049.
** Done cleansing data frames **


In [3]:
# Monthly POS By Sector 
df = pd.read_excel("./input/Monthly_Bulletin_September2021.xlsx",
                   sheet_name='30d', engine='openpyxl', skiprows=14)
columns_name = {'الفترة': 'Date'}
df.rename(columns=columns_name, inplace=True)
df.drop(df.columns[[0,2, 18, 35]], axis=1, inplace=True)
df.dropna(axis = 0, how = 'all', inplace = True)
df.dropna(axis = 1, how = 'all', inplace = True)
df.drop(df.tail(4).index, inplace = True)
df.drop(df.index[[0,1,2]], inplace=True)
df.reset_index(inplace=True, drop=True)
df.drop(df.index[0:27], inplace=True)
df.reset_index(inplace=True, drop=True)
#by anonymouse hero  
columnsTitles = ['Date', 'Number of Transactions', 'Value of Transactions']
Sector_final = pd.DataFrame(columns=columnsTitles)
sector_list_en = ['Transportation','Health','Restaurants & Café','Hotels','Beverage and Food','Clothing and Footwear',
                'Recreation and Culture','Miscellaneous Goods and Services','Electronic & Electric Devices ','Furniture',
                'Construction & Building Materials','Jewelry','Telecommunication','Education','Public Utilities','Others']
sector_list_ar = ['المواصلات','الصحة','المطاعم والمقاهي','الفنادق','المشروبات والأطعمة','الملابس والأحذية','الترفيه والثقافة','خدمات وسلع متنوعة*','الأجهزه الإلكترونية والكهربائية','الأثاث','مواد البناء و التعمير','المجوهرات ','الاتصالات','التعليم','المنافع العامة','أخرى']
x = 1
y = 2

for i, j in zip(sector_list_en, sector_list_ar):
    initial = df.iloc[:, [0, x,y]]

    mapping = {initial.columns[1]: 'Number of Transactions',
               initial.columns[2]: 'Value of Transactions'}
    initial = initial.rename(columns=mapping)

    initial['English_Sector'] = i
    initial['Arabic_Sector'] = j
    x = x + 2
    y = y + 2
    Sector_final = Sector_final.append(initial)
Sector_final.to_csv("./output/Monthly_pos_by_Sector.csv", index = False)

# Monthly POS By City 
df = pd.read_excel("./input/Monthly_Bulletin_September2021.xlsx",
                   sheet_name='30e', engine='openpyxl', skiprows=14)
df.dropna(axis = 0, how = 'all', inplace = True)
df.dropna(axis = 1, how = 'all', inplace = True)
df.drop(df.index[[0]], inplace=True)
df.drop(df.index[2:30], inplace=True)
df.dropna(axis = 1, how = 'all', inplace = True)
df.drop(df.columns[[16, 32]], axis=1, inplace=True)
columns_name = {'الفترة': 'Date'}
df.rename(columns=columns_name, inplace=True)
df.drop(df.index[[0, 1]], inplace=True)
#by anonymouse hero  
columnsTitles = ['Date', 'Number of Transactions', 'Value of Transactions','Number of Terminals']
City_final = pd.DataFrame(columns=columnsTitles)
city_list_en = ['Riyadh', 'Jeddah','Dammam', 'AL-Madinah','Makkah','Buraidah','Tabuk','Hail',
             'Abha','Jazan','Najran','Skaka','Arar','AL-Bahah']
city_list_ar = ['الرياض', 'جدة','الدمام', 'المدينة المنورة','مكة المكرمة','بريدة','تبوك','حائل',
             'أبها','جازان','نجران','سكاكا','عرعر','الباحة']
x = 1
y = 2
z = 3

for i, j in zip(city_list_en, city_list_ar):
    initial = df.iloc[:, [0, x,y,z]]

    mapping = {initial.columns[1]: 'Number of Transactions',
               initial.columns[2]: 'Value of Transactions', 
               initial.columns[3]: 'Number of Terminals' }
    initial = initial.rename(columns=mapping)

    initial['English_city'] = i
    initial['Arabic_City'] = j
    x = x + 3
    y = y + 3
    z = z + 3
    City_final = City_final.append(initial)


# Add latitude & longitude for map chart using Nominatim
group_City = City_final.groupby(by='English_city').agg({'Value of Transactions' : 'sum', 'Number of Transactions' : 'sum'})
group_City = group_City.reset_index()
location = [x for x in group_City['English_city'].unique().tolist() 
            if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    # remove things that does not seem usefull here
    try:
        address = location[i] + ', Saudi Arabia'
        geolocator = Nominatim(user_agent="sa_explorer@gmail.com")
        loc = geolocator.geocode(address)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    except:
        # in the case the geolocator does not work, then add nan element to list
        # to keep the right size
        latitude.append(np.nan)
        longitude.append(np.nan)
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'English_city':location, 
                    'location_latitude': latitude,
                    'location_longitude':longitude})
# merge on English_City with Groupe_City to get the column 
Grouped_City = group_City.merge(df_, on='English_city', how='left')
City_final = pd.merge(City_final, Grouped_City[['English_city', 'location_latitude', 'location_longitude']], on='English_city')
City_final.to_csv("./output/Monthly_pos_by_citites.csv", index = False)